In [1]:
import os
import glob
import ast
import json
import pickle
import numpy as np
import seaborn as sns
import pandas as pd
# from dance_evaluation import *
import matplotlib.pyplot as plt
# import analyze_hits_pos as anlyz

### Analyze Hits from multiple sensors

In [2]:
config1 = {"a": 60, "b": [100,110,120,130,140,150,160,170,180,190,200], "mode": ["uni", "bi"],
           "win_size": 5}

configs = [config1]

bpm_dict = {
    "bpm_avg_x": 0, "bpm_avg_y": 1, "bpm_avg_xy": 2, "bpm_mode_x": 3,
    "bpm_mode_y": 4, "bpm_mode_xy": 5, "bpm_median_x": 6, "bpm_median_y": 7,
    "bpm_median_xy": 8
}

# Load genre mappings
with open("genreID_count_mapping.json", "r") as file:
    genre_Tcount = json.load(file)

with open("genre_symbols_mapping.json", "r") as file:
    genre_name = json.load(file)

tol = 6
tol_type = "rel"
case = "tempo_cases"
foot1s_name = "right_ankle"

for cfg in configs:
    a = cfg["a"]
    w_sec = cfg['win_size']
    h_sec = w_sec/2
    
    for mode in cfg["mode"]:
        for b in cfg['b']:

            foot_1S_df = pd.read_csv(f"./saved_results/{case}/tempo_{a}_{b}/pos/foot/{foot1s_name}_W{w_sec}H{h_sec}_zero_{mode}_{a}_{b}.csv")
            total = foot_1S_df.shape[0]             # total number of 
            json_hits_data = {}

            for key in bpm_dict.keys():

                #################### POSITION #######################
                pos_left_hand_score = pd.read_csv(f"./saved_results/{case}/tempo_{a}_{b}/pos/score/hand/left_wrist_W{w_sec}H{h_sec}_{tol_type}{tol}_{mode}_{a}_{b}_score.csv")
                pos_right_hand_score = pd.read_csv(f"./saved_results/{case}/tempo_{a}_{b}/pos/score/hand/right_wrist_W{w_sec}H{h_sec}_{tol_type}{tol}_{mode}_{a}_{b}_score.csv")

                pos_left_ankle_score = pd.read_csv(f"./saved_results/{case}/tempo_{a}_{b}/pos/score/foot/left_ankle_W{w_sec}H{h_sec}_{tol_type}{tol}_{mode}_{a}_{b}_score.csv")
                pos_right_ankle_score = pd.read_csv(f"./saved_results/{case}/tempo_{a}_{b}/pos/score/foot/right_ankle_W{w_sec}H{h_sec}_{tol_type}{tol}_{mode}_{a}_{b}_score.csv")
                
                pos_left_hand_score["hits_idx"] = pos_left_hand_score["hits_idx"].apply(ast.literal_eval)   # convert string to list
                pos_right_hand_score["hits_idx"] = pos_right_hand_score["hits_idx"].apply(ast.literal_eval)

                pos_left_ankle_score["hits_idx"] = pos_left_ankle_score["hits_idx"].apply(ast.literal_eval)
                pos_right_ankle_score["hits_idx"] = pos_right_ankle_score["hits_idx"].apply(ast.literal_eval)

                pos_L_hand_posvel_hits_idx = set(pos_left_hand_score.iloc[bpm_dict[key]]["hits_idx"])  # left hand
                pos_R_hand_posvel_hits_idx = set(pos_right_hand_score.iloc[bpm_dict[key]]["hits_idx"]) # right hand

                pos_L_foot_posvel_hits_idx = set(pos_left_ankle_score.iloc[bpm_dict[key]]["hits_idx"]) # left foot
                pos_R_foot_posvel_hits_idx = set(pos_right_ankle_score.iloc[bpm_dict[key]]["hits_idx"])    # right foot 

                pos_all_hits_idx = set().union(*[pos_L_hand_posvel_hits_idx, pos_R_hand_posvel_hits_idx, pos_L_foot_posvel_hits_idx, pos_R_foot_posvel_hits_idx])

                #################### VELOCITY ######################
                vel_left_hand_score = pd.read_csv(f"./saved_results/{case}/tempo_{a}_{b}/vel/score/hand/left_wrist_W{w_sec}H{h_sec}_{tol_type}{tol}_{mode}_{a}_{b}_score.csv")
                vel_right_hand_score = pd.read_csv(f"./saved_results/{case}/tempo_{a}_{b}/vel/score/hand/right_wrist_W{w_sec}H{h_sec}_{tol_type}{tol}_{mode}_{a}_{b}_score.csv")

                vel_left_ankle_score = pd.read_csv(f"./saved_results/{case}/tempo_{a}_{b}/vel/score/foot/left_ankle_W{w_sec}H{h_sec}_{tol_type}{tol}_{mode}_{a}_{b}_score.csv")
                vel_right_ankle_score = pd.read_csv(f"./saved_results/{case}/tempo_{a}_{b}/vel/score/foot/right_ankle_W{w_sec}H{h_sec}_{tol_type}{tol}_{mode}_{a}_{b}_score.csv")

                vel_left_hand_score["hits_idx"] = vel_left_hand_score["hits_idx"].apply(ast.literal_eval)   # convert string to list
                vel_right_hand_score["hits_idx"] = vel_right_hand_score["hits_idx"].apply(ast.literal_eval)

                vel_left_ankle_score["hits_idx"] = vel_left_ankle_score["hits_idx"].apply(ast.literal_eval)
                vel_right_ankle_score["hits_idx"] = vel_right_ankle_score["hits_idx"].apply(ast.literal_eval)

                vel_L_hand_posvel_hits_idx = set(vel_left_hand_score.iloc[3]["hits_idx"])  # left hand
                vel_R_hand_posvel_hits_idx = set(vel_right_hand_score.iloc[3]["hits_idx"]) # right hand

                vel_L_foot_posvel_hits_idx = set(vel_left_ankle_score.iloc[3]["hits_idx"]) # left foot
                vel_R_foot_posvel_hits_idx = set(vel_right_ankle_score.iloc[3]["hits_idx"])    # right foot

                vel_all_hits_idx = set().union(*[vel_L_hand_posvel_hits_idx, vel_R_hand_posvel_hits_idx, vel_L_foot_posvel_hits_idx, vel_R_foot_posvel_hits_idx])

                # zero velocity hit idx and peak velocity hits idx merged limb-wise
                L_hand_posvel_hits_idx = pos_L_hand_posvel_hits_idx.union(vel_L_hand_posvel_hits_idx)
                R_hand_posvel_hits_idx = pos_R_hand_posvel_hits_idx.union(vel_R_hand_posvel_hits_idx)

                L_foot_posvel_hits_idx = pos_L_foot_posvel_hits_idx.union(vel_L_foot_posvel_hits_idx)
                R_foot_posvel_hits_idx = pos_R_foot_posvel_hits_idx.union(vel_R_foot_posvel_hits_idx)

                posvel_all_hits_idx = set().union(*[L_hand_posvel_hits_idx, R_hand_posvel_hits_idx, L_foot_posvel_hits_idx, R_foot_posvel_hits_idx])

                hits_stats = {
                    
                    "hits_summary": {"segment": ["Left Hand", "Right Hand", "Left Foot", "Right Foot", "Combined"],
                                    "hits_idx": [L_hand_posvel_hits_idx, R_hand_posvel_hits_idx, L_foot_posvel_hits_idx, R_foot_posvel_hits_idx, posvel_all_hits_idx],
                                    "hits_count": [len(L_hand_posvel_hits_idx), len(R_hand_posvel_hits_idx), len(L_foot_posvel_hits_idx), len(R_foot_posvel_hits_idx), len(posvel_all_hits_idx)],
                                    "total": [total] * 5,            # total remains same for all
                                    "hits_percentage": [len(L_hand_posvel_hits_idx) / total * 100,
                                                        len(R_hand_posvel_hits_idx) / total * 100,
                                                        len(L_foot_posvel_hits_idx) / total * 100,
                                                        len(R_foot_posvel_hits_idx) / total * 100,
                                                        len(posvel_all_hits_idx) / total * 100]
                                    },
                    "hit_idx_by_limb": {"Left Hand": L_hand_posvel_hits_idx, "Right Hand": R_hand_posvel_hits_idx,
                                                            "Left Foot": L_foot_posvel_hits_idx,"Right Foot": R_foot_posvel_hits_idx,
                                                            "Combined": posvel_all_hits_idx},
                            }

                json_hits_data[key] = hits_stats
            save_path1 = f"./saved_results/{case}/tempo_{a}_{b}/stats_posvel/{mode}/hit_stats_W{w_sec}H{h_sec}_{tol_type}{tol}_{mode}_{a}_{b}.pkl"
            
            with open(save_path1, "wb") as f:
                pickle.dump(json_hits_data, f)